Computing Project - Monte carlo Simulation of fluids

In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import json
import re

In [ ]:
#load config parameters
def load_jsonc(path):
    with open(path, "r", encoding="utf-8") as f:
        text = f.read()

    # remove // comments
    text = re.sub(r"//.*", "", text)
    # remove /* block comments */
    text = re.sub(r"/\\*.*?\\*/", "", text, flags=re.DOTALL)

    return json.loads(text)

config = load_jsonc("config.jsonc")
print(config)


{'N': 100, 'eta': 0.68, 'sigma': 0.3, 'dim': 2, 'equil_steps': 50000, 'prod_steps': 500000, 'sample_every': 1000, 'tune_every': 1000, 'r_max': 5.0, 'gr_bins': 200, 'lattice': 'square', 'seed': 12345}


In [ ]:
#Computing box size from N, density, and particle diameter. Maximum particle diameter of 1? Why not define box as sqrt(N)?
def sigma_from_eta_2d(N: int, L: float, eta: float) -> float:
    """
    From 2D packing fraction:
      eta = N * (pi * sigma^2 / 4) / L^2
      -> sigma = 2 L sqrt(eta / (N pi))
    """
    return 2.0 * L * math.sqrt(eta / (N * math.pi))